# Logging
(back to the overview [offline](../main.ipynb),[online](https://nbviewer.jupyter.org/github/QCoDeS/Qcodes/tree/master/docs/examples/main.ipynb))

[read on nbviewer](https://nbviewer.jupyter.org/github/QCoDeS/Qcodes/tree/master/docs/examples/logging/logger.ipynb)

# TL;DR
 * There is a QCoDeS logging module: `qcodes.utils.logger`
 * Call `logger.start_all_logging` at the start of every script/session to make sure all log messages get stored to the `.qcodes/logs` folder in your home directory.
 * For debugging purposes you can log messages of an individual instrument (and see the VISA dialog).
 * You can obtain all logging messages in a `pandas.DataFrame` for further analysis.

# Contents
- [Introduction](#Introduction)
- [Set up logging in QCoDeS](#Set-up-logging-in-QCoDeS)
  - [QCoDeS default handlers](#QCoDeS-default-handlers)
  - [IPython command history](#IPython-command-history)
- [Temporarily elevating the logging level](#Temporarily-elevating-the-logging-level)
- [Filtering log messages by instrument](#Filtering-log-messages-by-instrument)
- [Capturing Pandas.DataFrame](#Capturing-Pandas.DataFrame)


# Introduction
Python offers an excellent logging framework that is widely used in QCoDeS. As a user you might be interested in reading the log messages that are generated by QCoDeS for debugging and also profiling purposes.

All log records(=messages+meta data) are created using a python `logging.logger` object. In QCoDeS there is a logger for every module with the same name as the module itself, i.e. you can use `logging.getLogger('qcodes.instrument_drivers.Lakeshore.Model_372')` for getting the logger of the Lakeshore temperature controller. From the dots in the names of the loggers a hierarchy is define, at the top of which is the so called *root logger* that catches all messages.
The log records created in a logger are processed by the *handlers* that have been attached to this specific *logger*.
The handlers can for example be used to output the log messages into different locations, like the console or a log file.

Every logger, handler and log record has an associated level. The default levels are: `CRITICAL` `ERROR`, `WARNING`, `INFO`, `DEBUG`. The levels determine how messages are passed: A logger passes only records that exceeds its minimal passing level to its handlers and the handlers in turn only output those messages that exceed their minimal passing level.


For a more detailed description of Python logging refer to the [official manual](https://docs.python.org/3.6/library/logging.html#module-logging) (This is all well summed up [in this chart](https://docs.python.org/3.6/howto/logging.html#logging-flow))

# Set up logging in QCoDeS
At the beginning of every experiment script file you should include the following lines:

In [1]:
from qcodes.logger import start_all_logging
start_all_logging()

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\a-dovoge\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\a-dovoge\anaconda3\lib\logging\__init__.py", line 992, in emit
    msg = self.format(record)
  File "C:\Users\a-dovoge\anaconda3\lib\logging\__init__.py", line 838, in format
    return fmt.format(record)
  File "C:\Users\a-dovoge\anaconda3\lib\logging\__init__.py", line 578, in format
    s = self.formatMessage(record)
  File "C:\Users\a-dovoge\anaconda3\lib\logging\__init__.py", line 547, in formatMessage
    return self._style.format(record)
  File "C:\Users\a-dovoge\anaconda3\lib\logging\__init__.py", line 391, in format
    return self._fmt % record.__dict__
KeyError: 'instrument_name'
Call stack:
  File "C:\Users\a-dovoge\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\a-dovoge\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\a-dovoge\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    ap

The call to `start_all_logging` does two things:
1. Create handlers to output logging messages from the root logger to the console and to a log file.
2. Enable logging of the ipython commands into a separate log file.

## QCoDeS default handlers
The effect of the console handler can be seen in the two printed lines with a red background below the previous code cell. Every line corresponds to a log record. The fields of the records are separated by `¦` and have the following meaning:
 0. time stamp
 1. logger name, here the module name
 2. record log level
 3. name of the function that created the log message
 4. line at which the message has been issued
 5. actual log message

The same format is used for the file handler. By default QCoDeS logs to the file `<user directory>/.qcodes/logs/qcodes.log`. To avoid endlessly long files, the files roll over at midnight and get the date appended to their name where `qcodes.log` without date always gives you the log messages of the same day.

To configure the levels you want to log/handle, you need to edit the `qcodesrc.json` file in you home directory. The default values are:
```
    "logger":{
        "console_level": "WARNING",
        "file_level": "INFO",
        "logger_levels":{
            "pyvisa": "INFO"
        }
```
While `console_level` and `file_level` describe the levels of the two default handlers described previously, `logger_levels` is a dictionary that can be used to limit the messages passed on by module level loggers. This means with these defaults, the pyvisa module will not log messages on a `DEBUG` level. Therefore setting the `console_level` to `DEBUG` will not show the pyvisa debug messages (you certainly don't want to see them as they log every single character that is passed to an instrument with an individual message). On the other hand setting the console level to warning will still suppress pyvisa info messages. 

## IPython command history
The output above that follows these logging messages comes from IPython. It tells us that all issued IPython commands will be logged to a file. The file lives in the same log directory as the python logs and is called `command_history.log`. It will not roll over.

To change the command history logging directory you will need to call `start_command_history_logger` with the new path as an argument.

# Temporarily elevating the logging level
Sometimes you might wish to catch e.g. all debug messages for a few lines of code. You can do this simply by:

In [2]:
import logging
import qcodes.logger as logger
log = logging.getLogger('example_logger')

log.debug('This message will not be visible as the logging level is set to `DEBUG`')

with logger.console_level(logging.DEBUG):
    log.debug('This message is visible as the logging level is temporarily elevated to `DEBUG`')
    
log.debug('The level is back to what it used to be')

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\a-dovoge\anaconda3\lib\logging\__init__.py", line 992, in emit
    msg = self.format(record)
  File "C:\Users\a-dovoge\anaconda3\lib\logging\__init__.py", line 838, in format
    return fmt.format(record)
  File "C:\Users\a-dovoge\anaconda3\lib\logging\__init__.py", line 578, in format
    s = self.formatMessage(record)
  File "C:\Users\a-dovoge\anaconda3\lib\logging\__init__.py", line 547, in formatMessage
    return self._style.format(record)
  File "C:\Users\a-dovoge\anaconda3\lib\logging\__init__.py", line 391, in format
    return self._fmt % record.__dict__
KeyError: 'instrument_name'
Call stack:
  File "C:\Users\a-dovoge\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\a-dovoge\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\a-dovoge\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    ap

For other handlers than the console handler you can do the same thing using the `handler_level` context manager.

# Filtering log messages by instrument
A very common use case for logging in qcodes is to analyze the communication with a VISA instrument. For this purpose the logger module provides a context manager to filter the logging messages by instrument.

Generally one could of course simply attach a handler to the logger of `qcodes.instrument_drivers.Vendor.Type` but this would not show the messages that are generated by the visa communication in `qcodes.instrument.visa`. Additionally with the approach implemented here one can not only filter messages for a given type of instrument but also of a specific instance.

To demonstrate this consider the following mock up of the AMI430 Magnet controller that consists of individual visa instruments for each axis, here with the names `x`, `y` and `z`:

In [3]:
from qcodes.instrument.ip_to_visa import AMI430_VISA
from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430_3D, AMI430Warning

import qcodes.instrument.sims as sims

visalib = sims.__file__.replace('__init__.py', 'AMI430.yaml@sim')

mag_x = AMI430_VISA('x', address='GPIB::1::INSTR', visalib=visalib,
                        terminator='\n', port=1)
mag_y = AMI430_VISA('y', address='GPIB::2::INSTR', visalib=visalib,
                        terminator='\n', port=1)
mag_z = AMI430_VISA('z', address='GPIB::3::INSTR', visalib=visalib,
                        terminator='\n', port=1)

import numpy as np
field_limit = [
    lambda x, y, z: x == 0 and y == 0 and z < 3,
    lambda x, y, z: np.linalg.norm([x, y, z]) < 2
]

driver = AMI430_3D("AMI430-3D", mag_x, mag_y, mag_z, field_limit)


2018-10-19 14:49:24,792 ¦ qcodes.instrument.base.com.visa ¦ INFO ¦ visa ¦ set_address ¦ 115 ¦ x ¦ [x(AMI430_VISA)] Opening PyVISA Resource Manager with visalib: c:\users\a-dovoge\qcodes\qcodes\instrument\sims\AMI430.yaml@sim
2018-10-19 14:49:25,276 ¦ qcodes.instrument.base.com.visa ¦ INFO ¦ visa ¦ set_address ¦ 124 ¦ x ¦ [x(AMI430_VISA)] Opening PyVISA resource at address: GPIB::1::INSTR
2018-10-19 14:49:25,286 ¦ qcodes.instrument.base.com.visa ¦ INFO ¦ visa ¦ set_address ¦ 115 ¦ y ¦ [y(AMI430_VISA)] Opening PyVISA Resource Manager with visalib: c:\users\a-dovoge\qcodes\qcodes\instrument\sims\AMI430.yaml@sim
2018-10-19 14:49:25,287 ¦ qcodes.instrument.base.com.visa ¦ INFO ¦ visa ¦ set_address ¦ 124 ¦ y ¦ [y(AMI430_VISA)] Opening PyVISA resource at address: GPIB::2::INSTR
2018-10-19 14:49:25,304 ¦ qcodes.instrument.base.com.visa ¦ INFO ¦ visa ¦ set_address ¦ 115 ¦ z ¦ [z(AMI430_VISA)] Opening PyVISA Resource Manager with visalib: c:\users\a-dovoge\qcodes\qcodes\instrument\sims\AMI430.ya

Connected to: QCoDeS AMI430_simulation (serial:1337, firmware:0.0.01) in 0.49s
Connected to: QCoDeS AMI430_simulation (serial:1337, firmware:0.0.01) in 0.01s
Connected to: QCoDeS AMI430_simulation (serial:1337, firmware:0.0.01) in 0.02s


In the above messages you can see the prefix `[<instrument_name>(<instrument_type>)]`. To analyze the visa communication one can use `filter_instrument`:

In [4]:
driver.cartesian((0, 0, 0))
with logger.console_level('DEBUG'):
    with logger.filter_instrument(mag_x):
        driver.cartesian((0, 0, 1))

2018-10-19 14:49:25,385 ¦ qcodes.instrument.base.com.visa ¦ DEBUG ¦ visa ¦ ask_raw ¦ 228 ¦ x ¦ [x(AMI430_VISA)] Querying: STATE?
2018-10-19 14:49:25,387 ¦ qcodes.instrument.base.com.visa ¦ DEBUG ¦ visa ¦ ask_raw ¦ 230 ¦ x ¦ [x(AMI430_VISA)] Response: 2
2018-10-19 14:49:25,398 ¦ qcodes.instrument.base.com.visa ¦ DEBUG ¦ visa ¦ ask_raw ¦ 228 ¦ x ¦ [x(AMI430_VISA)] Querying: FIELD:MAG?
2018-10-19 14:49:25,400 ¦ qcodes.instrument.base.com.visa ¦ DEBUG ¦ visa ¦ ask_raw ¦ 230 ¦ x ¦ [x(AMI430_VISA)] Response: 0
2018-10-19 14:49:25,403 ¦ qcodes.instrument.base.com.visa ¦ DEBUG ¦ visa ¦ ask_raw ¦ 228 ¦ x ¦ [x(AMI430_VISA)] Querying: FIELD:MAG?
2018-10-19 14:49:25,406 ¦ qcodes.instrument.base.com.visa ¦ DEBUG ¦ visa ¦ ask_raw ¦ 230 ¦ x ¦ [x(AMI430_VISA)] Response: 0


The output on the console shows as expected only messages from the `x` instrument. For multiple instruments pass a sequence of instruments to `filter_instrument`:

In [5]:
driver.cartesian((0, 0, 0))
with logger.console_level('DEBUG'):
    with logger.filter_instrument((mag_x, mag_y)):
        driver.cartesian((0, 0, 1))

2018-10-19 14:49:25,448 ¦ qcodes.instrument.base.com.visa ¦ DEBUG ¦ visa ¦ ask_raw ¦ 228 ¦ x ¦ [x(AMI430_VISA)] Querying: STATE?
2018-10-19 14:49:25,450 ¦ qcodes.instrument.base.com.visa ¦ DEBUG ¦ visa ¦ ask_raw ¦ 230 ¦ x ¦ [x(AMI430_VISA)] Response: 2
2018-10-19 14:49:25,452 ¦ qcodes.instrument.base.com.visa ¦ DEBUG ¦ visa ¦ ask_raw ¦ 228 ¦ y ¦ [y(AMI430_VISA)] Querying: STATE?
2018-10-19 14:49:25,453 ¦ qcodes.instrument.base.com.visa ¦ DEBUG ¦ visa ¦ ask_raw ¦ 230 ¦ y ¦ [y(AMI430_VISA)] Response: 2
2018-10-19 14:49:25,461 ¦ qcodes.instrument.base.com.visa ¦ DEBUG ¦ visa ¦ ask_raw ¦ 228 ¦ x ¦ [x(AMI430_VISA)] Querying: FIELD:MAG?
2018-10-19 14:49:25,464 ¦ qcodes.instrument.base.com.visa ¦ DEBUG ¦ visa ¦ ask_raw ¦ 230 ¦ x ¦ [x(AMI430_VISA)] Response: 0
2018-10-19 14:49:25,467 ¦ qcodes.instrument.base.com.visa ¦ DEBUG ¦ visa ¦ ask_raw ¦ 228 ¦ y ¦ [y(AMI430_VISA)] Querying: FIELD:MAG?
2018-10-19 14:49:25,469 ¦ qcodes.instrument.base.com.visa ¦ DEBUG ¦ visa ¦ ask_raw ¦ 230 ¦ y ¦ [y(AMI430

# Capturing Pandas.DataFrame
To process the logs, especially with the timestamps the [pandas module](https://pandas.pydata.org/) is highly suited. With `logger.log_to_dataframe` or `logger.logfile_to_dataframe` you can convert a log(file) to a pandas `DataFrame`. See this ([online](https://nbviewer.jupyter.org/github/QCoDeS/Qcodes/tree/master/docs/examples/logging/logfile_parsing.ipynb), [offline](logfile_parsing_ipynb)) notebook for an example.

You can also use a context manager to capture the logs directly into a `DataFrame`

In [6]:
with logger.console_level(logging.WARN):
    driver.cartesian((0, 0, 0))
    with logger.capture_dataframe(level='DEBUG') as (handler, get_dataframe):
        driver.cartesian((0, 0, 1))
        df = get_dataframe()
        driver.cartesian((0, 0, 2))
        df2 = get_dataframe() # this is the cummulative log

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\a-dovoge\anaconda3\lib\logging\__init__.py", line 992, in emit
    msg = self.format(record)
  File "C:\Users\a-dovoge\anaconda3\lib\logging\__init__.py", line 838, in format
    return fmt.format(record)
  File "C:\Users\a-dovoge\anaconda3\lib\logging\__init__.py", line 578, in format
    s = self.formatMessage(record)
  File "C:\Users\a-dovoge\anaconda3\lib\logging\__init__.py", line 547, in formatMessage
    return self._style.format(record)
  File "C:\Users\a-dovoge\anaconda3\lib\logging\__init__.py", line 391, in format
    return self._fmt % record.__dict__
KeyError: 'instrument_name'
Call stack:
  File "C:\Users\a-dovoge\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\a-dovoge\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\a-dovoge\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    ap

  File "C:\Users\a-dovoge\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-f22acd052feb>", line 4, in <module>
    driver.cartesian((0, 0, 1))
  File "c:\users\a-dovoge\qcodes\qcodes\instrument\parameter.py", line 288, in __call__
    self.set(*args, **kwargs)
  File "c:\users\a-dovoge\qcodes\qcodes\instrument\parameter.py", line 466, in set_wrapper
    set_function(raw_value, **kwargs)
  File "c:\users\a-dovoge\qcodes\qcodes\utils\command.py", line 177, in __call__
    return self.exec_function(*args)
  File "c:\users\a-dovoge\qcodes\qcodes\instrument_drivers\american_magnetics\AMI430.py", line 790, in _set_setpoints
    set_point.get_components("x", "y", "z")
  File "c:\users\a-dovoge\qcodes\qcodes\instrument_drivers\american_magnetics\AMI430.py", line 704, in _adjust_child_instruments
    log.debug("Field values OK, proceeding")
Message: 'Field values OK, proceeding'
A

  File "C:\Users\a-dovoge\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\a-dovoge\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\a-dovoge\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\a-dovoge\anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\a-dovoge\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Users\a-dovoge\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\a-dovoge\anaconda3\lib\asyncio\base_events.py", line 422, in run_forever
    self._run_once()
  File "C:\Users\a-dovoge\anaconda3\lib\asyncio\base_events.py", line 1432, in _run_once
    handle._run()
  File "C:\Users\a-dovoge\anaconda3\lib\asyncio

In [7]:
df

,asctime,name,levelname,module,funcName,lineno,instrument_name,message
0,"2018-10-19 14:49:25,616",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,228,x,[x(AMI430_VISA)] Querying: STATE?
1,"2018-10-19 14:49:25,617",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,230,x,[x(AMI430_VISA)] Response: 2
2,"2018-10-19 14:49:25,617",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,228,y,[y(AMI430_VISA)] Querying: STATE?
3,"2018-10-19 14:49:25,617",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,230,y,[y(AMI430_VISA)] Response: 2
4,"2018-10-19 14:49:25,617",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,228,z,[z(AMI430_VISA)] Querying: STATE?
5,"2018-10-19 14:49:25,618",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,230,z,[z(AMI430_VISA)] Response: 2
6,"2018-10-19 14:49:25,633",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,228,x,[x(AMI430_VISA)] Querying: FIELD:MAG?
7,"2018-10-19 14:49:25,633",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,230,x,[x(AMI430_VISA)] Response: 0
8,"2018-10-19 14:49:25,634",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,228,y,[y(AMI430_VISA)] Querying: FIELD:MAG?
9,"2018-10-19 14:49:25,634",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,230,y,[y(AMI430_VISA)] Response: 0


In [8]:
df2

,asctime,name,levelname,module,funcName,lineno,instrument_name,message
0,"2018-10-19 14:49:25,616",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,228,x,[x(AMI430_VISA)] Querying: STATE?
1,"2018-10-19 14:49:25,617",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,230,x,[x(AMI430_VISA)] Response: 2
2,"2018-10-19 14:49:25,617",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,228,y,[y(AMI430_VISA)] Querying: STATE?
3,"2018-10-19 14:49:25,617",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,230,y,[y(AMI430_VISA)] Response: 2
4,"2018-10-19 14:49:25,617",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,228,z,[z(AMI430_VISA)] Querying: STATE?
5,"2018-10-19 14:49:25,618",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,230,z,[z(AMI430_VISA)] Response: 2
6,"2018-10-19 14:49:25,633",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,228,x,[x(AMI430_VISA)] Querying: FIELD:MAG?
7,"2018-10-19 14:49:25,633",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,230,x,[x(AMI430_VISA)] Response: 0
8,"2018-10-19 14:49:25,634",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,228,y,[y(AMI430_VISA)] Querying: FIELD:MAG?
9,"2018-10-19 14:49:25,634",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,230,y,[y(AMI430_VISA)] Response: 0


You can of course combine the context managers like this:

In [9]:
with logger.console_level(logging.WARN):
    driver.cartesian((0, 0, 0))
    with logger.capture_dataframe(level='DEBUG') as (handler, get_dataframe):
        with logger.filter_instrument(mag_x, handler=handler):
            driver.cartesian((0, 0, 1))
            df = get_dataframe()
df

,asctime,name,levelname,module,funcName,lineno,instrument_name,message
0,"2018-10-19 14:49:26,142",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,228,x,[x(AMI430_VISA)] Querying: STATE?
1,"2018-10-19 14:49:26,143",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,230,x,[x(AMI430_VISA)] Response: 2
2,"2018-10-19 14:49:26,145",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,228,x,[x(AMI430_VISA)] Querying: FIELD:MAG?
3,"2018-10-19 14:49:26,146",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,230,x,[x(AMI430_VISA)] Response: 0
4,"2018-10-19 14:49:26,147",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,228,x,[x(AMI430_VISA)] Querying: FIELD:MAG?
5,"2018-10-19 14:49:26,148",qcodes.instrument.base.com.visa,DEBUG,visa,ask_raw,230,x,[x(AMI430_VISA)] Response: 0


For an analysis of the timestamps please also refer to the log analysis example notebook ([online](https://nbviewer.jupyter.org/github/QCoDeS/Qcodes/tree/master/docs/examples/logging/logfile_parsing.ipynb), [offline](logfile_parsing_ipynb)).